# SVD

The idea of singular value decomposition is to decompose a matrix $M$ into three matricies, $U$, $S$, $V$:

![title](../figs/svd.png)

Where $U$ and $V$ are orthogonal and $S$ contains the relative importance of each factor (i.e. the singular values).

For many application, we are only interested in a subset of the singular values ($k$ of them). Computing the full svd and taking the top $k$ values are usually inefficient.

Randomized SVD allow us to compute the $k$ singular values faster. Also for many statistical problems the data might be noise / incomplete and computing the exact SVD is overkill.

For a matrix $M$ of dimension $m \times n$ the SVD has a computational cost of $O(nm * min(n,m))$. For randomized SVD where we want $k$ singular values, the computational is $O(nmk)$, where $n, m \gg k$.

In [1]:
import numpy as np
from scipy import linalg
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=remove)

In [3]:
tfidf = TfidfVectorizer(stop_words='english')
X = tfidf.fit_transform(newsgroups.data) # (documents, vocab)
X = X.todense()
X = np.array(X)

In [4]:
def randomized_range_finder(A, size, n_iter=5):
    """
        Computes an orthonormal matrix Q whose range approximates the range of A.
        Can use:
            - Power iteration (fast but unstable).
            - LU (tradeof of the two other approaches).
            - QR (slow but most accurate).
    """

    Q = np.random.normal(size=(A.shape[1], size))

    for i in range(n_iter):
        Q, _ = linalg.lu(A @ Q, permute_l=True)
        Q, _ = linalg.lu(A.T @ Q, permute_l=True)

    Q, _ = linalg.qr(A @ Q, mode='economic')
    return Q

In [5]:
def randomized_svd(A, n_components, n_oversamples=10, n_iter=4):
    n_random = n_components + n_oversamples

    Q = randomized_range_finder(A, n_random, n_iter)

    B = Q.T @ A

    U_hat, s, V = linalg.svd(B, full_matrices=False)
    del B
    U = Q @ U_hat

    return U[:, :n_components], s[:n_components], V[:n_components, :]

In [6]:
np.random.seed(42)
%time u, s, v = randomized_svd(X, 5)

CPU times: user 1.34 s, sys: 6.46 ms, total: 1.35 s
Wall time: 353 ms


In [7]:
%time u_full, s_full, v_full = linalg.svd(X, full_matrices=False)

CPU times: user 32.2 s, sys: 654 ms, total: 32.8 s
Wall time: 8.7 s


In [8]:
print("Singular values for randomized SVD:")
print(s)
print()
print("Singular values for full SVD:")
print(s_full[0:5])

Singular values for randomized SVD:
[5.19203756 3.54422516 3.07251962 2.84901097 2.68890078]

Singular values for full SVD:
[5.19203791 3.54430061 3.0736553  2.85311835 2.69921543]
